In [1]:
# import dependencies
import random as rn
from simpful import *
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")
import string
import pickle

## Importing the machine learning model for sentiment analysis

In [2]:
# natural language processing block
import spacy
nlp = spacy.load('en_core_web_sm')
import string
punct = string.punctuation
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS) # list of stopwords

# creating a function for data cleaning
def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
 
  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:    # Stopwords and punctuation removal
      cleaned_tokens.append(token)
  return cleaned_tokens

In [3]:
# sentiment analysis model
import pickle
# Load the saved model using pickle
with open('emotion_detector.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Defining the HSTN

In [4]:
class hstn():
    
    # initiating the interaction counter
    interaction_counter = 0
    # initiating user engagement
    user_input = []
    currentSuperStateTransition = False
    userEngagement = 0
    userOpinion_positive = []
    nextSuperState = 2 # 0: supporting user; 1: listening to user; 2: main task
    number_of_interactions_fixed = 6

    # rules
    # first system looks for 6 interactions
    # after the a superstate breaks, switches to other superstate then, it willl looks for next three interactions before making IM decision 

    # Main state - Greeting
    def greeting(self):
        greeting_responses = ["Hi!","Good morning","Hey, what's up?","Greetings!","How are you doing?","Hi, there! How's your day going?",
                              "Hey, nice to see you!","Well, hello to you too!","Hi, how can I assist you today?","Hello, friend!",
                              "How's everything?","Hey, it's great to see you again!","Hi there, what's new?","How's your day been so far?",
                              "Hello, how's life treating you?"]
        print (greeting_responses[rn.randint(0,len(greeting_responses)-1)])

    # super state - about lecture
    def about_lecture(self, sentence):
        about_lec = sentence
        #print (about_lec)
        print (about_lec)
        return about_lec 
    
    # super state - task organization
    def task_organization(self, sentence):
        # superstate - Task organization
        task_org_responses = sentence
        return task_org_responses
    
    # superstate - Ask about student interaction
    def student_interaction(self, sentence):
        # superstate - Student Interaction
        student_int_responses = sentence
        return student_int_responses
    
    # superstate - Ask about Clarity 
    def clarity(self, sentence):
        # superstate ask about Clarity of the lecture
        clarity_responses = sentence
        return clarity_responses
    
    # listeing to user
    def listening_to_user(self, sentence):
        listening_response = sentence
        return listening_response

    # supporting to user
    def Supporting_user(self, sentence):
        supporting_user = sentence
        return supporting_user
    
    # Main state - Concluding
    def concluding(self):
        concluding_responses = ["Thank you for your insights and valuable input. Your perspective is greatly appreciated", "I appreciate your time and thoughtful responses. Thank you for sharing your thoughts with me",
                                "Thank you for taking the time to discuss these matters with me. Your input has been instrumental", "I'm grateful for the opportunity to hear your thoughts on this. Thank you for your time and thoughtful contributions",
                                "Your time and expertise are highly valued. Thank you for engaging in this discussion with me", "Thank you for spending your valuable time sharing your experiences and opinions. It has been a meaningful conversation",
                                "I want to express my gratitude for your time and the enriching conversation we've had. Thank you for your insights", "Thank you for your thoughtful responses. Your time and input have been invaluable to this discussion",
                                "I appreciate the depth of our conversation. Thank you for dedicating your time and sharing your perspective with me", "Thank you for your time and openness in discussing these matters. It has been a pleasure engaging in this conversation with you"]
        print (concluding_responses[rn.randint(0,len(concluding_responses)-1)])

    # calulating the fuzzy output value considering emotional cue and user engagement as input
    def calc_fuzzy(self, val1, val2):
        # A simple fuzzy inference system for the tipping problem
        # A simple fuzzy inference system for the tipping problem
        # Create a fuzzy system object
        FS = simpful.FuzzySystem()

        # Define fuzzy sets and linguistic variables
        S_1 = simpful.FuzzySet(points=[[0., 1.], [0.19, 1.], [0.49, 0.0]], term='Negative')
        S_2 = simpful.FuzzySet(points=[[0.203 , 0.], [0.503 , 1.], [0.8023, 0.]], term="Neutral")
        S_3 = simpful.FuzzySet(points=[[0.5011, 0.], [0.798 , 1.], [1., 1.]], term="Positive")
        FS.add_linguistic_variable("Emotion", simpful.LinguisticVariable([S_1, S_2, S_3], concept="Recent Emotion", universe_of_discourse=[0,1]))

        F_1 = simpful.FuzzySet(points=[[0., 1.], [0.097, 1.0], [0.2974, 0]], term="Very_low")
        F_2 = simpful.FuzzySet(points=[[0.157, 0.0], [0.34, 1.0], [0.5043, 0.0]], term="Low")
        F_3 = simpful.FuzzySet(points=[[0.4, 0.0], [0.5, 1], [0.6008, 0.0]], term="Medium")
        F_4 = simpful.FuzzySet(points=[[0.5, 0.0], [0.758, 1.0], [0.851, 0.0]], term="High")
        F_5 = simpful.FuzzySet(points=[[0.702, 0.0], [0.9009, 1], [1.0, 1.0]], term="Very_High")
        FS.add_linguistic_variable("Engagement", simpful.LinguisticVariable([F_1, F_2, F_3, F_4, F_5], concept="User Engagement", universe_of_discourse=[0,1]))

        # Define output crisp values
        FS.set_crisp_output_value("Supporting", 0.1)
        FS.set_crisp_output_value("Listening", 0.5)
        FS.set_crisp_output_value("Main", 1.0)

        # Define function for generous tip (food score + service score + 5%)
        #FS.set_output_function("generous", "Food+Service+5")

        # Define fuzzy rules
        R1 = "IF (Emotion IS Negative) AND (Engagement IS Very_low) THEN (Tip IS Supporting)"
        R2 = "IF (Emotion IS Negative) AND (Engagement IS Low) THEN (Tip IS Supporting)"
        R3 = "IF (Emotion IS Negative) AND (Engagement IS Medium) THEN (Tip IS Supporting)"
        R4 = "IF (Emotion IS Negative) AND (Engagement IS High) THEN (Tip IS Listening)"
        R5 = "IF (Emotion IS Negative) AND (Engagement IS Very_High) THEN (Tip IS Listening)"
        R6 = "IF (Emotion IS Neutral) AND (Engagement IS Very_low) THEN (Tip IS Supporting)"
        R7 = "IF (Emotion IS Neutral) AND (Engagement IS Low) THEN (Tip IS Supporting)"
        R8 = "IF (Emotion IS Neutral) AND (Engagement IS Medium) THEN (Tip IS Listening)"
        R9 = "IF (Emotion IS Neutral) AND (Engagement IS High) THEN (Tip IS Main)"
        R10 = "IF (Emotion IS Neutral) AND (Engagement IS Very_High) THEN (Tip IS Main)"
        R11 = "IF (Emotion IS Positive) AND (Engagement IS Very_low) THEN (Tip IS Supporting)"
        R12 = "IF (Emotion IS Positive) AND (Engagement IS Low) THEN (Tip IS Supporting)"
        R13 = "IF (Emotion IS Positive) AND (Engagement IS Medium) THEN (Tip IS Listening)"
        R14 = "IF (Emotion IS Positive) AND (Engagement IS High) THEN (Tip IS Main)"
        R15 = "IF (Emotion IS Positive) AND (Engagement IS Very_High) THEN (Tip IS Main)"
        FS.add_rules([R1, R2, R3,R4,R5,R6,R7,R8,R9,R10,R11,R12,R13,R14,R15])


        # Set antecedents values
        FS.set_variable("Emotion", val1)
        FS.set_variable("Engagement", val2)

        # Perform Sugeno inference and print output
        return(FS.Sugeno_inference(["Tip"]))
    
    # defining a function for calculating user engagement
    def calc_user_eng(self, array):
        # assuming array consists of all the user responses
        last_six_int = array[-6:]
        # total words in last 6 interactions
        total_words = 0
        # a way if getting total number of words in recent 6 interactions
        for i in last_six_int:
            words = i.split()
            total_words = total_words + len(words)
        # getting number of long responses
        long_response = 0
        for i in last_six_int:
            if len(i.split()) >= (total_words / 6):
                long_response = long_response + 1
        return (long_response / 6)
    
    # defining a function for calculating user opinon
    def calc_user_opn(self, array):
        # assuming array consist of all of the user opnions for their responses
        last_six_int = array[-6:]
        # the above have a mix of 'positive' and 'negative'
        pos_opn = 0
        for i in last_six_int:
            if i == "positive":
                pos_opn = pos_opn + 1
        return (pos_opn / 6)

    # a method to loop through all sentences in a list
    def loop_all(self, sentence, loaded_model):
        #print (type(sentence))
        if len(sentence) > 0:
            for i in sentence:
                print (i)
                user_input_instant = self.get_user_input()
                opinion = loaded_model.predict([user_input_instant])
                print (opinion)
                self.userOpinion_positive.append(opinion)

                self.interaction_counter = self.interaction_counter + 1
                if (self.interaction_counter) >= self.number_of_interactions_fixed:
                    # once the number of interaction exceeded 6 then system looks for next 3 interactions before changing interaction mode
                    #self.number_of_interactions_fixed = 6
                    
                    # getting interaction modes
                    self.state_switch(self.userOpinion_positive, self.user_input, self.currentSuperStateTransition)

                    # decison making based on interaction modes
                    if (self.nextSuperState == 2):
                        pass

                    elif (self.nextSuperState == 1):
                        self.interaction_counter = 0
                        self.loop_all(self.listening_to_user, loaded_model)

                    elif (self.nextSuperState == 0):
                        self.interaction_counter = 0
                        self.loop_all(self.Supporting_user, loaded_model)

                    elif (self.nextSuperState == 3):
                        break

                    else:
                        pass        
        else:
            # ignoring the empty list 
            pass

    # defining state switching condition
    def state_switch(self, userOpinion, userEngagement, currentSuperstateTransition):
        userOpinion = self.calc_user_opn(userOpinion)
        print (userOpinion)
        userEngagement = self.calc_user_eng(userEngagement)
        print (userEngagement)
        interactionMode = self.calc_fuzzy(userOpinion, userEngagement)
        interactionMode = interactionMode['Tip']
        if currentSuperstateTransition == False:

            if interactionMode <= 0.32:
                self.nextSuperState = 0
                self.currentSuperStateTransition = True
            elif interactionMode <= 0.74:
                self.nextSuperState = 1
                self.currentSuperStateTransition = True
            else:
                self.nextSuperState = 2
        else:
            if interactionMode <= 0.74:
                self.nextSuperState = 3 # 3: denotes to break from the current super state and switch to the next direct superstate
                self.currentSuperStateTransition = False
                self.interaction_counter = 3
                
            else:
                self.nextSuperState = 2 # 2: go ahead with the main task

    # run the overall system
    def run_system(self):
        self.greeting()
        self.get_user_input()
        self.loop_all(self.about_lecture, loaded_model)
        self.loop_all(self.task_organization, loaded_model)
        self.loop_all(self.student_interaction, loaded_model)
        self.loop_all(self.clarity, loaded_model)
        self.concluding()

    # get the user input
    def get_user_input(self):
        user_input_instant = input("Type your response here: ")
        self.user_input.append(user_input_instant)
        return user_input_instant

    # getting the user responses at once
    def get_user_response(self):
        return (self.user_input)


In [5]:
text1 = ["What were the main key points or takeaways from today's lecture?","Can you summarize the lecture in your own words?",
                            "Were there any concepts or ideas discussed in the lecture that you found challenging or unclear? If so, please explain.",
                            "How do you think the material covered in this lecture relates to previous lectures or topics we've discussed in the course?",
                            "Can you provide an example or real-life application of one of the concepts discussed in the lecture?",
                            "Did the lecturer mention any open questions or areas of research related to the topic? What are your thoughts on those?",
                            "How would you apply what you've learned in today's lecture to solve a specific problem or scenario?",
                            "Were there any interesting or surprising insights from the lecture that you'd like to share or discuss further?",
                            "Can you identify any connections between the content of this lecture and current events or trends in the field?",
                            "Do you have any questions or concerns about the material that was covered today?"]

text2 = ["Speed of the sessions/time management were reasonable","Relevant course matter was provided",
                                "Recommended useful textbooks, websites, periodicals","Syllabus was substantially covered in the class",
                                "Number of worked examples and tutorials were adequate","The lecturer promotes self studies by the student",
                                "Practical applications relevant to the module were discussed","Professional approach of the lecturer was high",
                                "The lecturer advised regarding evaluation","Continuous assessment helps the learning process",
                                "Feedback on continuous assessment was helpful to identify my weaknesses before he final examination"]
text3 = ["To what extent did the lecturer facilitate and encourage active discussions among students?", "How effective was the lecturer in creating an environment conducive to student participation?",
                                 "How frequently did the lecturer recognize and appreciate student responses?", "How well did the lecturer manage the balance between encouraging participation and maintaining order?",
                                 "How actively did the lecturer encourage students to ask questions?", "How constructive and helpful was the feedback given to students during discussions?",
                                 "How well did the lecturer adapt their teaching based on student responses?", "What strategies did the lecturer employ to enhance student engagement during discussions?",
                                 "How effective were these strategies in promoting a dynamic and engaging learning environment?", "ow accessible and approachable did the lecturer appear during interactions with students?",
                                 "How did the lecturer balance individual and group interactions for good learning experience?"]
text4 = ["Was the pace of the delivery consistent throughout the session?", "Did the lecturer maintain an appropriate speed, allowing for easy understanding?",
                             "How effective were the black/white board or PowerPoint presentations in conveying information?", "Could you easily hear and understand the lecturer throughout the session?",
                             "How clear were the verbal explanations provided by the lecturer?", "Were complex concepts articulated in a way that was easy to comprehend?",
                             "How effectively did the lecturer address questions to ensure understanding?", "How responsive was the lecturer to feedback provided by students?",
                             "How well did the lecturer accommodate diverse learning preferences?"]
text5 = ["I agree with your opinion", "Please share more about your opinion, that will be useful for future students",
                             "feel free to talk about any inconvenience you have"]
text6 = ["Please don't miss out any lectures each and every lectures will be valuable", "At least you will learn something in the lecture rather than avoiding it",
                          "In lecutres try to concentrate on what is explained", "You can view the lecture materials before the lecture is conducted then you can understand more on lecutre itself"]


In [6]:
# creating an object of hstn class
dialogue_system = hstn()

In [7]:
# asking for greeting
print ("This will be greeting")
dialogue_system.greeting()
print ("This will be conclusion")
dialogue_system.concluding()

This will be greeting
How are you doing?
This will be conclusion
Your time and expertise are highly valued. Thank you for engaging in this discussion with me


In [8]:
# storing the text1 in about_lecture method and others for building the diaogue system
dialogue_system.about_lecture = text1
dialogue_system.clarity = []#text4
dialogue_system.student_interaction = []#text3
dialogue_system.task_organization = []#text2
dialogue_system.listening_to_user = text5
dialogue_system.Supporting_user = text6

In [9]:
# running the overall system
dialogue_system.run_system()

Hi!
What were the main key points or takeaways from today's lecture?
['positive']
Can you summarize the lecture in your own words?
['negative']
Were there any concepts or ideas discussed in the lecture that you found challenging or unclear? If so, please explain.
['negative']
How do you think the material covered in this lecture relates to previous lectures or topics we've discussed in the course?
['positive']
Can you provide an example or real-life application of one of the concepts discussed in the lecture?
['positive']
Did the lecturer mention any open questions or areas of research related to the topic? What are your thoughts on those?
['positive']
0.6666666666666666
0.5
  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type
I agree with 

In [10]:
# List of strings
strings_list = dialogue_system.get_user_response()
print (strings_list)

# Open a file named "output.txt" in write mode
with open("output.txt", "w") as file:
    # Write each string from the list to the file
    for string in strings_list:
        file.write(string + "\n")



['hello', 'all were good and abour electrical circuits', 'yeah its all about DC circuits', 'yeah the concpets were very well explained by the professor', 'they were awesome', 'yeah the dc transformer analysis is a good example for real life application', 'no that is a drawback in the lecture', 'okay that sounds good', 'yeah if lecture explains more about research then it could have been interesting', 'okay sure thanks for your concern', "for that i don't have an idea", "this was a basic lecture i coudn't get any ", 'no question i have']


In [11]:
dialogue_system.calc_fuzzy(0.5, 0.5)

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type


{'Tip': 0.4896979461777583}

#### Code to make a textfile from recorded user2 responses

In [12]:
new_response_list = ['hello', "today's lecture was about solving linear systems of equations, the gaussian elimination method was explained well in lecture", 'yeah as I said above it is about solving linear systems of equation using gaussian elimination method', 'yeah the concpets were explained well by lecturer, they were all clearly explained', 'this topic included some knowledge from advanced levels but they are very wimple to understand', 'many numercial algorithms in our computer are solved by thi technique, it is really excited to know this', 'yeah, he asked question about our awareness in space research where these kind of numerical algorithms are extensively used. it was quite inspiring', 'okay thank you', 'yeah the lecture was quite intersting', 'yeah sure but now i am fine', 'in our day to day problems i can apply these concepts to solve sensor calibration for instance', 'yeah the talk about space research  was really fascinating', 'yeas recent example of peregrine mission was a really good example', 'yeah the speed was good and i was able to follow the lecture well', 'okay thank you', 'yeah, the space mission related explanations provided by the professor were really interesting and inspiring', 'no i am good', 'yeah they were provided and ther are the good sources for learning', 'yeah they were also provided, i feel they are good for learning as well again', 'yeah the professor was really good at covering everything in the lecture itself', 'yeah they were awesome, example were very excellent', 'yeah he guides the student for self study and it is very inspiring', 'yeah practicall applications were very well described and  they really worth to learn', 'yeah it was really good, professor was very professional, i like that', 'yeah all matters regarding evaluation was given in the class prior to evaluatiousually professor promotes discussions in class and they were really interesting', 'okay sure i will keep this in my mind', 'yeah that is true', 'yeah definitely', 'yah this is a goo sharing what they think and get valuable feedbacks from professor as well as fellow students', 'yeah pace was good', 'okay fine', 'yeah true', 'yes definitely', 'yeah good advice',  'yes speed is okay', 'they were well desigined and they serve the purpose correctly']
print (f"There are {len(new_response_list)} responses")

# Open a file named "output.txt" in write mode
with open("output3.txt", "w") as file:
    # Write each string from the list to the file
    for string in new_response_list:
        file.write(string + "\n")

There are 36 responses


In [15]:
# first interaction mode change
dialogue_system.calc_fuzzy(0.666, 0.5)

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type


{'Tip': 0.48030625301384566}

In [14]:
# second interaction mode change
dialogue_system.calc_fuzzy(0.5, 0.5)

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type


{'Tip': 0.4896979461777583}

In [16]:
# third interaction mode change
dialogue_system.calc_fuzzy(0.5, 0.5)

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type


{'Tip': 0.4896979461777583}

In [17]:
# fourth interaction mode change
dialogue_system.calc_fuzzy(0.83333, 0.666)

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type


{'Tip': 1.0}

In [18]:

dialogue_system.calc_fuzzy(0.8333, 0.8333)

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type


{'Tip': 1.0}

In [19]:
dialogue_system.calc_fuzzy(0.83333, 0.16666)

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type


{'Tip': 0.09999999999999999}

In [20]:
# interaction mmode change
dialogue_system.calc_fuzzy(0.6666, 0.16666)

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type


{'Tip': 0.09999999999999999}

In [21]:
dialogue_system.calc_fuzzy(0.5, 0.3333)

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type


{'Tip': 0.1}

In [23]:
dialogue_system.calc_fuzzy(0.6666, 0.5)

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type


{'Tip': 0.4803065531890307}

In [24]:
response_list = ["Hello! Life's treating me well, thank you for asking. How can I assist you today?", "The main key points from today's lecture on machine learning with Professor Bishop included understanding various algorithms, their applications, and practical implementations.", 'The lecture with Professor Bishop covered a wide range of topics in machine learning, from fundamental algorithms to their real-world applications. It provided a balanced mix of theory and practical insights, making it an enriching learning experience.', 'No, I found the concepts discussed in the lecture to be clear and understandable.', "The material covered in this lecture builds upon the foundational concepts we've previously discussed in the course. It  extends our understanding of machine learning algorithms and their applications, providing deeper insights into the field.", 'One real-life application discussed in the lecture is the use of machine learning algorithms in healthcare for diagnosing diseases based on medical imaging data, such as X-rays or MRI scans.', 'Yes, the lecturer mentioned some open questions and areas of research related to machine learning, such as improving the interpretability and explainability of complex models, dealing with biased data, and enhancing the robustness of algorithms against adversarial attacks. These are crucial areas for further exploration as they address some of the current limitations and challenges in the field, ultimately advancing the development and deployment of machine learning solutions in various domains.', "Certainly, I'll make sure not to miss out on any lectures, as each one provides valuable insights and knowledge essential for a comprehensive understanding of the course material.", 'Absolutely, attending every lecture is essential for continuous learning and growth.  Each lecture offers opportunities to gain new knowledge, deepen understanding, and explore different perspectives, all of which contribute to personal and intellectual development.', "Of course, concentrating on the explanations provided during lectures is crucial for maximizing learning outcomes. I'll make sure to stay focused and absorb the information presented to enhance my understanding of the subject matter.", "That's a great suggestion! Reviewing lecture materials beforehand can indeed provide a solid foundation and better prepare me to grasp the concepts discussed during the lecture. I'll make sure to utilize this approach to enhance my understanding further.", "I would apply the concepts learned in today's lecture on machine learning to address a problem such as customer churn prediction for a subscription-based service. By employing classification algorithms taught in the lecture and utilizing historical customer data, I could develop a predictive model to identify customers at risk of churning. This would enable proactive measures to retain customers and optimize business strategies.", 'One interesting insight from the lecture was the discussion on the ethical considerations of using machine learning algorithms, particularly regarding issues of bias and fairness. It was intriguing to delve into how biases in data can perpetuate societal inequalities and the importance of mitigating these biases in algorithmic decision-making processes. This topic sparked further reflection on the ethical implications of deploying machine learning systems in various contexts and the need for responsible AI practices.', 'Yes, the speed of the sessions and time management were reasonable. The lecturer effectively covered the planned topics within the allocated time, allowing for thorough explanations and ample opportunities for student engagement and discussion.', 'Yes, relevant course material was provided during the lecture, covering key concepts and topics essential for understanding machine learning principles and applications.', 'The lecturer recommended several useful resources for further study, including textbooks like "Pattern Recognition and Machine Learning" by Christopher M. Bishop and "Introduction to Statistical Learning" by Gareth James et al. Additionally, websites like Towards Data Science and periodicals such as the Journal of Machine Learning Research were suggested for staying updated on the latest research and developments in the field of machine learning.', 'Thank you! If you have any more  questions or need further clarification, feel free to ask.', "Certainly! My opinion is based on the satisfaction I derived from the lecture experience. The content was well-organized and covered relevant topics in machine learning. The lecturer's approach to teaching, including clear explanations and effective time management, contributed to a positive learning  environment. Additionally, the recommended resources provided valuable supplementary material for further exploration. Overall, I believe future students will find this lecture beneficial for building a strong foundation in machine learning.", "I didn't encounter any significant inconveniences during the lecture. However, if I were to suggest areas for improvement, it might be helpful to incorporate more interactive activities or practical demonstrations to reinforce learning. Additionally, providing additional resources or study materials  in advance could further enhance the learning experience for students.", 'Yes, the syllabus was substantially covered in the class. The lecture included discussions on various topics outlined in the syllabus, such as different machine learning algorithms, their applications, and ethical considerations. The content provided a comprehensive overview of the subject matter, aligning well with the course objectives.', 'Yes, the number of worked examples and tutorials provided during the lecture was adequate. They helped reinforce the theoretical concepts covered and allowed for practical application of the material, enhancing understanding and retention for students.', 'Yes, the lecturer promotes self-study by encouraging students to explore additional resources, such as textbooks, websites, and periodicals, to deepen their understanding of the subject matter. This emphasis on self-study fosters independent learning and allows students to delve deeper into areas of interest within the field of machine learning.', 'Yes, practical applications relevant to the module were extensively discussed during the lecture. The lecturer provided examples of how machine learning algorithms are applied in various real-world scenarios, such as healthcare, finance, and marketing. These practical applications helped illustrate the relevance and importance of the concepts covered in the module.', 'The lecturer actively facilitated and encouraged discussions among students throughout the lecture. They welcomed questions, encouraged participation, and fostered an interactive learning environment. This approach enhanced student engagement and allowed for the exchange of ideas and perspectives, enriching the learning experience for everyone.', 'The lecturer was highly effective in creating an environment conducive to  student participation. They encouraged questions, welcomed diverse viewpoints, and fostered a supportive atmosphere where students felt comfortable sharing their thoughts and engaging in discussions. This approach promoted active participation and enhanced the overall learning experience for everyone involved.', 'The lecturer consistently recognized and appreciated student responses throughout the lecture. They actively acknowledged contributions from students, providing positive feedback and validation for their input. This recognition helped foster a supportive and inclusive learning environment, encouraging continued engagement and participation from all students.', 'Thank you for your agreement! If you have any further questions or need additional information, feel free to ask.', "The lecture offered a good balance between theory and practical applications, making the concepts clear and engaging. Professor Bishop's explanations were well-organized and easy to follow. This would be helpful for future students who want a solid foundation in machine learning.", "No inconvenience to report! I was very satisfied with the lecture overall.  If anything comes to mind, I'll be sure to let you know. ", 'Professor Bishop struck a good balance between encouraging participation and maintaining order. The lecture felt interactive without being disruptive.  ', "Professor Bishop created a welcoming environment for questions. While I don't recall specific prompts to ask questions every minute, I felt comfortable raising my hand if something needed clarification. ", "I can't say for certain about the feedback during discussions as my notes don't mention that specifically.  However, since I was satisfied with the level of student interaction, it's likely Professor Bishop provided constructive feedback when needed. ", "Yes, the pace was likely consistent. As someone with good analytical skills, I would have noticed if the lecture significantly sped up or slowed down throughout. Since I don't recall any such issues, the pace was probably consistent. ", 'Yes. The lecture content was clear and explanations were well-organized, which suggests Professor Bishop maintained an appropriate speed for understanding.  ', "I can't recall for certain whether Professor Bishop used a blackboard, whiteboard, or PowerPoint during the lecture.  My focus was on the clarity of the explanations, not the specific presentation method.  However, since I was very satisfied with the lecture overall, the chosen method(s) must have been effective  in conveying the information.", "Thank you!  I'm glad to hear our perspectives align.   ", 'As someone who appreciates clear explanations, I found the lecture well-structured and easy to follow. This suggests Professor Bishop likely used effective visuals or presentation methods to complement their explanations. This can be helpful for future students who learn  best with a combination of auditory and visual information. ', "Like before, I didn't experience any major inconveniences during the lecture.  If anything arises in the future lectures, I'll be sure to mention them.  ", 'Yes. The lecture hall acoustics seemed good and Professor Bishop likely spoke clearly, because I had no trouble hearing or understanding them throughout the session. ', "Very clear.  As someone with good reasoning skills, I rely on precise explanations. Professor Bishop's verbal explanations were well-organized and easy to  follow, which significantly contributed to my positive learning experience. ", "Yes. The lecture balanced theory with practical applications, which likely helped break down complex concepts. Professor Bishop's clear explanations and well-organized delivery (as suggested by my previous points) would have further aided comprehension of these concepts. "]

In [27]:
print (response_list)

["Hello! Life's treating me well, thank you for asking. How can I assist you today?", "The main key points from today's lecture on machine learning with Professor Bishop included understanding various algorithms, their applications, and practical implementations.", 'The lecture with Professor Bishop covered a wide range of topics in machine learning, from fundamental algorithms to their real-world applications. It provided a balanced mix of theory and practical insights, making it an enriching learning experience.', 'No, I found the concepts discussed in the lecture to be clear and understandable.', "The material covered in this lecture builds upon the foundational concepts we've previously discussed in the course. It  extends our understanding of machine learning algorithms and their applications, providing deeper insights into the field.", 'One real-life application discussed in the lecture is the use of machine learning algorithms in healthcare for diagnosing diseases based on medic

In [28]:
# Open a file named "output.txt" in write mode
with open("user01.txt", "w") as file:
    # Write each string from the list to the file
    for string in response_list:
        file.write(string + "\n")

In [46]:
from simpful import *

# calulating the fuzzy output value considering emotional cue and user engagement as input
def calc_fuzzy(val1, val2):
    # A simple fuzzy inference system for the tipping problem
    # A simple fuzzy inference system for the tipping problem
    # Create a fuzzy system object
    FS = simpful.FuzzySystem()

    # Define fuzzy sets and linguistic variables
    S_1 = simpful.FuzzySet(points=[[0., 1.], [0.19, 1.], [0.49, 0.0]], term='Negative')
    S_2 = simpful.FuzzySet(points=[[0.203 , 0.], [0.503 , 1.], [0.8023, 0.]], term="Neutral")
    S_3 = simpful.FuzzySet(points=[[0.5011, 0.], [0.798 , 1.], [1., 1.]], term="Positive")
    FS.add_linguistic_variable("Emotion", simpful.LinguisticVariable([S_1, S_2, S_3], concept="Recent Emotion", universe_of_discourse=[0,1]))

    F_1 = simpful.FuzzySet(points=[[0., 1.], [0.097, 1.0], [0.2974, 0]], term="Very_low")
    F_2 = simpful.FuzzySet(points=[[0.157, 0.0], [0.34, 1.0], [0.5043, 0.0]], term="Low")
    F_3 = simpful.FuzzySet(points=[[0.4, 0.0], [0.5, 1], [0.6008, 0.0]], term="Medium")
    F_4 = simpful.FuzzySet(points=[[0.5, 0.0], [0.758, 1.0], [0.851, 0.0]], term="High")
    F_5 = simpful.FuzzySet(points=[[0.702, 0.0], [0.9009, 1], [1.0, 1.0]], term="Very_High")
    FS.add_linguistic_variable("Engagement", simpful.LinguisticVariable([F_1, F_2, F_3, F_4, F_5], concept="User Engagement", universe_of_discourse=[0,1]))

    # Define output crisp values
    FS.set_crisp_output_value("Supporting", 0.1)
    FS.set_crisp_output_value("Listening", 0.5)
    FS.set_crisp_output_value("Main", 1.0)

    # Define function for generous tip (food score + service score + 5%)
    #FS.set_output_function("generous", "Food+Service+5")

    # Define fuzzy rules
    R1 = "IF (Emotion IS Negative) AND (Engagement IS Very_low) THEN (Tip IS Supporting)"
    R2 = "IF (Emotion IS Negative) AND (Engagement IS Low) THEN (Tip IS Supporting)"
    R3 = "IF (Emotion IS Negative) AND (Engagement IS Medium) THEN (Tip IS Supporting)"
    R4 = "IF (Emotion IS Negative) AND (Engagement IS High) THEN (Tip IS Listening)"
    R5 = "IF (Emotion IS Negative) AND (Engagement IS Very_High) THEN (Tip IS Listening)"
    R6 = "IF (Emotion IS Neutral) AND (Engagement IS Very_low) THEN (Tip IS Supporting)"
    R7 = "IF (Emotion IS Neutral) AND (Engagement IS Low) THEN (Tip IS Supporting)"
    R8 = "IF (Emotion IS Neutral) AND (Engagement IS Medium) THEN (Tip IS Listening)"
    R9 = "IF (Emotion IS Neutral) AND (Engagement IS High) THEN (Tip IS Main)"
    R10 = "IF (Emotion IS Neutral) AND (Engagement IS Very_High) THEN (Tip IS Main)"
    R11 = "IF (Emotion IS Positive) AND (Engagement IS Very_low) THEN (Tip IS Supporting)"
    R12 = "IF (Emotion IS Positive) AND (Engagement IS Low) THEN (Tip IS Supporting)"
    R13 = "IF (Emotion IS Positive) AND (Engagement IS Medium) THEN (Tip IS Listening)"
    R14 = "IF (Emotion IS Positive) AND (Engagement IS High) THEN (Tip IS Main)"
    R15 = "IF (Emotion IS Positive) AND (Engagement IS Very_High) THEN (Tip IS Main)"
    FS.add_rules([R1, R2, R3,R4,R5,R6,R7,R8,R9,R10,R11,R12,R13,R14,R15])


    # Set antecedents values
    FS.set_variable("Emotion", val1)
    FS.set_variable("Engagement", val2)

    # Perform Sugeno inference and print output
    return(FS.Sugeno_inference(["Tip"]))

In [47]:
array1 = [[0.6666, 0.3333], [0.83333, 0.33333], [0.83333, 0.5], [1.0, 0.6666], [1.0, 0.16666], [1.0, 0.5], [1.0, 0.5], [1.0, 0.5], [1.0, 0.5]]

for i in array1:
    print(calc_fuzzy(i[0], i[-1]))

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type
{'Tip': 0.1}
  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type
{'Tip': 0.1}
  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type
{'Tip': 0.48979833926453153}
  ____  __  _  _  ____  ____  _  _  

In [48]:
print(calc_fuzzy(0.8333, 0.5))
print(calc_fuzzy(0.8333, 0.3333))

  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type
{'Tip': 0.48979833926453153}
  ____  __  _  _  ____  ____  _  _  __   
 / ___)(  )( \/ )(  _ \(  __)/ )( \(  ) v2.11.0 
 \___ \ )( / \/ \ ) __/ ) _) ) \/ (/ (_/\ 
 (____/(__)\_)(_/(__)  (__)  \____/\____/

 Created by Marco S. Nobile (m.s.nobile@tue.nl)
 and Simone Spolaor (simone.spolaor@unimib.it)

 * Detected Sugeno model type
{'Tip': 0.1}
